In [ ]:
#visualize hog features
'''fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8), sharex=True, sharey=True) 

ax1.imshow(resized_img, cmap=plt.cm.gray) 
ax1.set_title('Input image')

# Rescale histogram for better display 
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10)) 

ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray) 
ax2.set_title('Histogram of Oriented Gradients')

plt.show()'''

In [6]:
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import numpy as np
import os
%matplotlib inline
import glob

def img_hog(img):

    fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8), 
                        cells_per_block=(2, 2), visualize=True, multichannel=True)

    fd = np.reshape(fd, (1, -1))
    
    return fd

img_dir = "./Images"
img_path = os.path.join(img_dir,'*g')
files = glob.glob(img_path)
data = np.empty((0, 3780), int)

for file in files:
    print(file)
    img = imread(file)
    resized_img = resize(img, (128,64)) 
    #imshow(resized_img)
    res = img_hog(resized_img)
    data = np.append(data, res, axis=0)


./Images\Bahia_absinthifolia_close.jpg
./Images\Bahia_pedata.jpg
./Images\Bebbia_juncea.jpg
./Images\Bebbia_juncea_2.jpg
./Images\Bidens_aurea.jpg
./Images\Justicia_candicans.jpg


In [8]:
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(data)
distances, indices = nbrs.kneighbors(data)

print(distances)
print(indices)

[[0.         6.78189594 7.01577186]
 [0.         6.32545524 6.59194279]
 [0.         6.29842034 6.32545524]
 [0.         6.29842034 6.59194279]
 [0.         9.85681079 9.86549306]
 [0.         9.05257624 9.31486587]]
[[0 1 2]
 [1 2 3]
 [2 3 1]
 [3 2 1]
 [4 3 2]
 [5 1 3]]


In [47]:
from sklearn.decomposition import PCA
#n_components=0.80 means it will return the Eigenvectors that have the 80% of the variation in the dataset

pca = PCA(n_components=0.9)
pca.fit(data)
data_pca = pca.transform(data)

print(data_pca)

[[-2.04490254 -3.28611029 -3.2422714 ]
 [-2.58060309 -2.35962096  3.45464991]
 [ 6.65605231 -0.15484767  0.21339845]
 [-2.03054668  5.80057891 -0.42577695]]


In [2]:
import sqlite3
import json
import numpy as np
import math
import pandas
import cv2
import glob
from skimage.transform import downscale_local_mean
from skimage.feature import hog
import ntpath
from skimage import feature as ft
import skimage.feature
import os

def lbpimage(file):
    sqldbfile = 'mwdbSql1.sqlite'
    img = imread(file)
    k = 6
    data={}
    x={}
    y={}
    conn = sqlite3.connect(sqldbfile)
    c = conn.cursor()
    conn.commit()
    save = {}
    save_eu={}

    #RGB to GrayScale
    eps = 1e-7
    hist = []
    # Initializing LBP settings - radius and number of points
    radius = 3
    numPoints = 8 * radius
    gray = cv2.cvtColor(img, cv2.cv2.COLOR_BGR2GRAY)
    lbp = ft.local_binary_pattern(gray, numPoints, radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, numPoints + 3), range=(0, numPoints + 2))
    # normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + eps)

    img_fd = np.asarray(hist).tolist()

    # Retrieve all the Image Features except Query Image
    c.execute("SELECT * from imageslbp")
    rows = c.fetchall()

    # Calculate Distance Metric and Compare for k similar images
    for row in rows:
      fd_load = json.loads(row[1])
      fd = np.array(fd_load)
      data[row[0]]=fd
      score = chi2_distance(img_fd,data[row[0]])
      #distance= euclidean0_1(img_fd, data[row[0]])

      x[row[0]]=score
      #y[row[0]]=distance
    save = sorted(x.items(), key=lambda z: z[1])
    #save_eu = sorted(y.items(), key=lambda z: z[1])
    
    print("Chi Squared:")
    j=0

    res = []
    dets = pandas.read_csv(r".\pro_combined.csv", encoding='latin-1')
    for i in save:
        n = i.split('.')[0]
        ixd = int(n)
        row = dets.iloc[ixd]
        sim = 100 - int(100 / (1 + save[i]))
        if j < k:
            ele = {
                'imgname': n,
                'name': dets.iloc[ixd]['scientificName'],
                'genus': dets.iloc[ixd]['genus'],
                'country': dets.iloc[ixd]['country'],
                'state': dets.iloc[ixd]['stateProvince'],
                'county': dets.iloc[ixd]['county'],
                'value': sim
            }
            print(ele)
            res.append(ele)
        else:
            return res
        j=j+1

def chi2_distance(histA, histB, eps=1e-10):
    # compute the chi-squared distance
    d = 0.5 * np.sum([((a - b) ** 2) / (a + b + eps)
                      for (a, b) in zip(histA, histB)])

    # return the chi-squared distance
    return d

def euclidean0_1(vector1, vector2):

    d=math.sqrt(np.sum([((a-b) ** 2) for (a,b) in zip(vector1,vector2)]))
    return d
def hogimage():
    sqldbfile = 'mwdbSql1.sqlite'
    name = input("enter the name of the image file: ")
    k = int(input("enter k: "))
    data = {}
    x = {}
    y={}
    conn = sqlite3.connect(sqldbfile)
    c = conn.cursor()
    # img = cv2.imread("C://Users//hp//Desktop//MWDB//Hands//Hands//Few" + name)

    # Retrieve Query Image Features
    c.execute("SELECT * from imageshog where name= (?)", [name])
    fd_query = c.fetchall()
    fd_load1 = json.loads(fd_query[0][1])
    fd1 = np.array(fd_load1)
    print(fd1)
    conn.commit()
    save = {}
    save_eu={}

    # Retrieve all the Image Features except Query Image
    c.execute("SELECT * from imageshog where name != (?)", [name])
    rows = c.fetchall()
    for row in rows:
        fd_load = json.loads(row[1])
        fd = np.array(fd_load)

        # Calculate Distance Metric and Compare for k similar images
        data[row[0]] = fd
        score = chi2_distance(fd1, data[row[0]])
        distance = euclidean0_1(fd1, data[row[0]])
        x[row[0]] = score
        y[row[0]]= distance
    save = sorted(x.items(), key=lambda y: y[1])
    save_eu = sorted(y.items(), key=lambda z: z[1])
    print("Chi Squared:")
    j=0
    for i in save:

        if j<k:
            print(i)
        else:
            break;
        j=j+1
    print("Eucledian:")
    j=0
    for i in save_eu:

        if j<k:
            print(i)
        else:
            break;
        j=j+1



if __name__== "__main__":
    name = input("Which Model: 1) LBP 2)HOG ")
    path = os.path.join(os.getcwd(), 'images', '2.jpg');
    if(name== '1'):
        lbpimage(path)
    elif(name== '2'):
        hogimage()
    else:
        print("Wrong option")

ModuleNotFoundError: No module named 'cv2'